<center>
    <img src="../imagens/logo_APL.png" width="300" alt="APL logo"  />
</center>

# Pré-processamento dos Dados em `Python`

**Bem vindo!** Neste notebook serão discutidoas ferramentas para pré-processamento dos dados em `Python`. Ao final, espera-se que você seja capaz de entender parte do processo de transformação dos dados do formato inicial para um formato mais adequado para análise.

<h2>Conteúdo:</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li>Introdução</li>
    <li>Exemplo de Análise
        <ul>
            <li> Leitura do dataset </li>
            <li> Descrever características básicas dos dados </li>
        </ul>
    </li>
    <li>Agrupamento dos Dados </li>
    <li>Identificar e tratar valores ausentes </li>
    <li>Normalização de dados </li>
    <li>Data Binning </li>
</ul>
</div>


<hr>

<h2>Introdução</h2>

Esta introdução tem por objetivo converter os dados a partir de sua forma inicial para outros formatos, de maneira a preparar os dados para análises posteriores. 

A etapa de pré-processamento pode ser entendida como um conjunto de atividades que envolvem preparação, organização e estruturação dos dados. Sintetizar as principais características  e obter uma melhor compreensão dos dados.  

No nosso curso, para melhor organização, separamos essa etapa de pré-processamento da fase posterior que chamamos de Análise Exploratória de Dados. Entretanto, destacamos que essas duas etapas são interdependentes e acontecem em conjunto. E são de grande importância, pois será determinante para a qualidade final dos dados que serão analisados. 

Iniciamos carregando as bibliotecas que iremos utilizar:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

A linha `%matplotlib inline` faz parte da mágica do Jupyter para mostrar figuras ao longo do notebook python.

In [ ]:
%matplotlib inline

<h2>Exemplo de Análise</h2>


Na seção anterior vimos como manipular dados que foram criados durante esta apresentação, acontece que, na maioria das vezes, queremos analisar dados que já estão prontos.
A biblioteca `Pandas` nos fornece uma série de funcionalidades de leitura de dados, pros mais diversos formatos estruturais de dados, entre eles estão:
 1. `pd.read_csv`, para ler arquivos .csv, formato comum de armazenar dados de tabelas
 1. `pd.read_xlsx`, para ler arquivos Excel .xlsx, é necessário instalar uma biblioteca adicional pra esta funcionalidade.
 1. `pd.read_html`, para ler tabelas diretamente de um website
 
Usaremos para analisar dados externos nesta introdução o `.read_csv`, pois é neste formato que se encontram nossos dados. CSV, ou comma-separated values é um formato muito comum de dados abertos, trata-se, como a sigla sugere, de valores divididos por vírgula, apesar de o caracter separador poder ser o ponto-e-vírgula ou outro.

O arquivo `dados.csv` está na mesma pasta do nosso script, então podemos passar como argumento do `.read_csv` apenas o seu nome. Outro argumento interessante da função é o `sep`, que por padrão é a vírgula, mas que pode ser definido como outro caractere caso seu dado esteja usando outro separador.

Estes dados que usaremos como exemplo são dados sobre preços de apartamentos em 7 bairros da cidade do Rio de Janeiro: Botafogo, Copacabana, Gávea, Grajaú, Ipanema, Leblon, Tijuca. 

São dados adaptados de um arquivo que pode ser encontrado [aqui](https://raw.githubusercontent.com/mvinoba/notebooks-for-binder/master/dados.csv). Os campos em cada registro são listados a seguir:

| Nome        | Descrição                          |
| ----------- | ---------------------------        |
| condominio  | Valor (R\$) do Condomínio          |
| quartos     | Numero de quartos                  |
| suites      | Numero de suítes                   |
| vagas       | Numero de vagas de garagem         |
| area        | Tamanho da área construída (m2)    |
| bairro      | Nome do bairro                     |
| preco       | Preço (R\$) do imóvel              |
| pm2         | Preço (R\$) por metro quadrado     |


### Leitura do dataset

O nosso dataset é carregado por meio do comando `pd.read_csv()`, de acordo com:

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/APL-Data-Intelligence/AcelerAI/main/Curso_NEED/datasets/dados.csv")
print(df.head())

### Descrever características básicas dos dados

Como esperado, o DataFrame tem muitas linhas de dados, pra visualizar sucintamente as primeiras linhas de um DataFrame existe o método `.head()`

In [ ]:
df.head()

Por padrão `.head()` exibe as 5 primeiras linhas, mas isso pode ser alterado:

In [ ]:
df.head(n=10)

Similarmente existe o `.tail()`, que exibe por padrão as últimas 5 linhas do DataFrame:

In [ ]:
df.tail()

Podemos verificar os valores únicos de uma coluna usando o método `unique()`. A título de exemplo, vamos aplicar esse método na coluna relativa aos bairros:

In [ ]:
df["bairro"].unique()

Também parece interessante verificarmos a hegemoneidade da nossa amostra em relação aos bairros. Pra tarefas de contar valores podemos sempre aproveitar de outro método disponível, o `.value_counts()`, também veremos um pouco mais abaixo como visualizar estes valores em forma de gráfico de barras.

In [ ]:
df["bairro"].value_counts()

Os valores contados também podem ser normalizados para expressar porcentagens:

In [ ]:
df["bairro"].value_counts(normalize=True)

## Agrupamento dos Dados

Agrupar os dados se baseando em certos critérios é outro processo que o pandas facilita bastante com o `.groupby()`.
Esse método pode ser usado para resolver os mais **amplos** dos problemas, aqui abordarei apenas o agrupamento simples, a divisão de um DataFrame em grupos.

Abaixo agrupamos o nosso DataFrame pelos valores da coluna `"bairro"`, e em seguida aplicamos o `.mean()` para termos um objeto GroupBy com informação das médias agrupadas pelos valores da coluna bairros. 

In [ ]:
df.groupby("bairro").mean()

Para extrairmos dados de uma coluna deste objeto basta acessá-lo convencionalmente, para obtermos os valores da média do preço do metro quadrado em ordem crescente, por exemplo:

In [ ]:
df.groupby("bairro").mean()["pm2"].sort_values()

É comum queremos aplicar uma função qualquer aos dados, ou à parte deles, neste caso o pandas fornece o método `.apply`. Por exemplo, para deixar os nomes dos bairros como apenas as suas três primeiras letras:

In [ ]:
def truncar(bairro):
    return bairro[:3]

df["bairro"].apply(truncar).head()

Ou de um jeito mais prático, usando uma função `lambda`:

In [ ]:
df["bairro"].apply(lambda x: x[:3]).head()

## Identificar e tratar valores ausentes

Uma das tarefas na qual o pandas é reconhecidamente poderoso é a habilidade de tratar dados incompletos.
Por muitos motivos pode haver incompletude no dataset, o `np.nan` é um valor especial definido no Numpy, sigla para Not a Number, o pandas preenche células sem valores em um DataFrame lido com `np.nan`.

Vamos criar um novo dataframe usando as 5 primeiras linhas do nosso original, usando o já visto `.head()`. Abaixo é usado o `.replace` para substituir um valor específico por um `NaN`. 

In [ ]:
df2 = df.head()
df2 = df2.replace({"pm2": {12031.25: np.nan}})
df2

O pandas simplifica a remoção de quaiquer linhas ou colunas que possuem um `np.nan`, por padrão o `.dropna()` retorna as linhas que não contém um NaN:

In [ ]:
df2.dropna()

Preencher todos os valores NaN por um outro específico também é bastante simples:

In [ ]:
df2.fillna(99)

Acaba sendo muitas vezes conveniente termos um método que indica quais valores de um dataframe são NaN e quais não são:

In [ ]:
df2.isna()

## Normalização de dados

<b>Por que normalização?</b>

Normalização (padronização) é o processo de transformar valores de várias variáveis (em escalas de valores diferentes) para uma faixa única, comum a todas. As normalizações típicas incluem, mas não se limitam, a: i) dimensionar a variável para que sua média seja 0 e sua variância seja 1 ou ii) dimensionar a variável para que sua faixa de valores varie entre 0 a 1

<b>Exemplo</b>
<p>Para demonstrar o procedimento de normalização, usaremos a coluna "condominio", de forma que o valor normalizado varie entre 0 e 1. </p>

In [ ]:
# replace (original value) by (original value)/(maximum value)
df['condominio_normalizado'] = df['condominio']/df['condominio'].max()
df[['condominio', 'condominio_normalizado']]

## Data Binning

Conceito de _Data Binning_: pode ser entendido como o processo de transformar dados numéricos em variáveis categóricas (limitadas por faixas de valores), ou seja, um agrupamento dos dados numéricos em diferentes categorias.

O método `cut` do **Pandas** divide o conjunto de dados em variáveis categóricas (ou seja, _bins_)  limitadas por faixas de valores. No exemplo a seguir, foram utilizadas faixas de mesma largura. O método `value_counts` retorna uma tabela de frequência representando o número de ocorrências dentro de cada faixa de valores. Usaremos a feature que representa o custo (em R$) do condomínio.

In [ ]:
binnedPopulation = pd.cut(df['condominio'], 10)
print(binnedPopulation.value_counts())

**Interpretando esse resultado**: existem 839 residências em que o condomínio custa entre 708 e 1415 reais. Em 831 residências, esse custo é menor que 708 reais. Em 236, esse custo varia entre 1.415 e 2.122 reais. E assim sucessivamente...

O gráfico de **histograma ilustra graficamente** essa representação dos valores numéricos agrupados em faixas de valores.

In [ ]:
ax = (df['condominio']).plot.hist(bins=10, edgecolor='black', figsize=(4, 4))
ax.set_xlabel('Condomínio (R$)')
ax.set_ylabel('No de Ocorrências')
ax.set_xlim(0, 7000)

plt.tight_layout()
plt.show()

<hr>

## Direitos Autorais

[APL Data Intelligence](https://linktr.ee/APLdataintelligence)&#8482;  2021. Este notebook Python e seu código fonte estão liberados sob os termos da [Licença do MIT](https://bigdatauniversity.com/mit-license/).